# 9) How Can Stock Market Mitigate the Israeli-Palestinian Conflict?

[Vitor Kamada](https://www.linkedin.com/in/vitor-kamada-1b73a078)

E-mail: econometrics.methods@gmail.com

Last updated: 11-5-2020

"Commerce is a cure for the most destructive prejudices." (Montesquieu, 1748: Vol II, Ch 1)

Jha & Shayo (2019) randomly split 1345 Jewish Israeli voters between financial asset treatment and control groups. They reported that exposure to the stock market increases 4-6% the probability of voting for parties that favor a peaceful resolution of the conflict.
Let's open the dataset of Jha & Shayo (2019). Each row is a Israeli citizen.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('precision', 3)

# Data from Jha & Shayo (2019)
path = "https://github.com/causal-methods/Data/raw/master/" 
df = pd.read_stata(path + "replicationdata.dta")
df.head(5)

/usr/local/lib/python3.6/dist-packages/pandas/io/stata.py:1434: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  warnings.warn(msg, UnicodeWarning)


,stat,tradestock6,willingrisk1to10,uid,male,rel,relid,nafa,edu,ses,married,kids,vote2013,wave,age,age2,tradestock6all,litscore,patient,partyorder,prim_treatment,palstock,cash,na,assettreat,symbol,value,asset,stat_instructions,q5_determinants_m12,coded_r_learned_1,coded_r_learned_2,coded_r_learned_3,coarse_r_learned_1,coarse_r_learned_2,coarse_r_learned_3,vote2015,stimer_s3,starttime_s3,endtime_s3,...,tradedout_f26,tradedout_m4,tradedout_m12,tradedout_m19,tradedout_m26,ever_actout_preel,eng_z,party_s3,early_winner,early_loser,late_winner,late_loser,early_portgain,early_portloser,late_portgain,late_portloser,v_didnotvote,v_habayit,v_haamitanu,v_yisraelbeitenu,v_likud,v_yahaduthatorah,v_shas,v_kulanu,v_yeshatid,v_zu,v_meretz,v_uarabp,v_other,ddtotime_wkfin_m4,tradetot_m4,pricechange_m4,bought_bm4,sold_bm4,active_bm4,assettype,lastweek,last3yrs,nextweek,facts_0_m4
0,completed,NaN,3,60814.0,0.0,jewish,secular,jerusalem,MA,Below Average,0.0,0,other,1.0,51.0,2601.0,0.0,5.0,1.0,6.0,P_Lo,1.0,0.0,0.0,1.0,PALTEL,200.0,Palestine Telecommunications,completed,domestic political conditions: quality of gove...,Do not know,No label,No label,Not informative,Not informative,Not informative,hamkhane hazioni (avoda-tnuaa),53981.281,19-03-15 14:59,19-03-15 15:03,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.337,hamkhane hazioni (avoda-tnuaa),0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,7,3.0,-0.673,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0
1,completed,1.0,2,60824.0,0.0,jewish,secular,tel-aviv,BA,Above Average,1.0,1,kadimah,1.0,29.0,841.0,1.0,7.0,1.0,5.0,Cash_Hi,0.0,1.0,0.0,1.0,,400.0,,completed,"economic conditions: price stability, debt, qu...","Capital market behavior, investment patterns",No label,No label,General behavior and specifically in the capit...,Not informative,Not informative,hamkhane hazioni (avoda-tnuaa),78405.656,21-03-15 21:46,21-03-15 21:49,...,0.0,0.0,0.0,0.0,0.0,0.0,0.674,hamkhane hazioni (avoda-tnuaa),0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,10,3.0,5.323,2.0,0.0,2.0,1.0,1.0,1.0,0.0,3.0
2,completed,0.0,3,61067.0,1.0,jewish,secular,center,PhD,Above Average,1.0,2,yesh atid,2.0,38.0,1444.0,0.0,5.0,1.0,9.0,P_Hi,1.0,0.0,0.0,1.0,BOP,400.0,Bank of Palestine,completed,domestic political conditions: quality of gove...,Do not know,No label,No label,Not informative,Not informative,Not informative,hamkhane hazioni (avoda-tnuaa),54148.590,19-03-15 15:02,19-03-15 15:06,...,1.0,1.0,1.0,1.0,1.0,1.0,-0.235,hamkhane hazioni (avoda-tnuaa),0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6,3.0,0.000,2.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0
3,completed,NaN,4,61095.0,1.0,jewish,secular,haifa,BA student,Below Average,0.0,0,likud beyteynu,1.0,24.0,576.0,0.0,6.0,1.0,4.0,Cash_Hi,0.0,1.0,0.0,1.0,,400.0,,completed,domestic political conditions: quality of gove...,Identify potential private investors / marketi...,No label,No label,Characterizing investors,Not informative,Not informative,NaN,75867.422,21-03-15 21:04,21-03-15 21:08,...,0.0,0.0,0.0,NaN,NaN,0.0,0.226,hamkhane hazioni (avoda-tnuaa),1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,8,3.0,5.323,3.0,2.0,3.0,0.0,0.0,1.0,0.0,1.0
4,completed,0.0,4,61198.0,1.0,jewish,secular,north,MA,Below Average,0.0,0,meretz,1.0,45.0,2025.0,0.0,7.0,1.0,12.0,P_Lo,1.0,0.0,0.0,1.0,BOP,200.0,Bank of Palestine,completed,"economic conditions: price stability, debt, qu...",Confidence in Palestinian / Arab economies,No label,No label,Mentions of the Palestinian economy\capital ma...,Not informative,Not informative,meretz,57282.820,19-03-15 15:54,19-03-15 15:57,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.665,meretz,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,3.0,0.000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [2]:
# Drop missing values
df = df.dropna(subset=['left_s3'])

Graphic 1 shows that the control and treatment group voted similarly in the 2013 election. But in 2015, the left-wing party received 24.8% of the votes in the control group against 30.9% in the treatment group. In a reverse way, the right-wing party received 35.8% of the votes in the control group against 31.2% in the treatment group. According to Jha & Shayo (2019), both right and left parties are similar in terms of economic policies. The main difference is that the left party favors a peaceful process, while the right party sees that any concession for peace would put at risk the State of Israel.

In [3]:
# Data: Vote Share by year
v2013 = df.groupby('assettreat')['left_2013', 'right_2013'].mean().T
v2015 = df.groupby('assettreat')['left_s3', 'right_s3'].mean().T
prop = v2013.append(v2015)

# Plot Bar Chart
import plotly.graph_objects as go
node = ['Left 2013', 'Right 2013', 'Left 2015', 'Right 2015']

fig = go.Figure(data=[             
    go.Bar(name='Control', x=node, y = prop[0]),
    go.Bar(name='Treatment', x=node, y = prop[1]) ])

fig.update_layout(barmode='group',
  title_text = 'Graphic 1 - Elections: 2013 vs 2015 ',
  font=dict(size=18) )

fig.update_yaxes(title_text = "Vote Share")

fig.show()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


The table below shows that the treatment group is similar to the control group. The exceptions are age and willingness to take risks. Israelites in the treatment group are younger than the control group (39.3 vs. 41.5 years old). The treatment group also has a higher preference to take risks evaluated in an index that varies from 1 to 10 (4.7 vs. 4.3). 

We calculate the p-values based on OLS regressions with strata fixed effects.

In [4]:
control = ['right_2013', 'left_2013', 'p_index_s1', 'e_index_init',
    'tradestock6all', 'male', 'age', 'postsecondary', 'BA_student',
	  'college_grad', 'married', 'r_sec',  'r_trad', 'r_relig', 'r_ultra', 
		'g_jerusalem', 'g_north', 'g_haifa', 'g_center', 'g_telaviv', 'g_south',
    'g_wb', 'faminc', 'willingrisk1to10', 'patient', 'plitscore']

In [5]:
import statsmodels.formula.api as smf

result = []
for var in control:
    # OLS with 104 randomization strata fixed effects
    reg = smf.ols(var + "~ 1 + assettreat + C(block13)", df)
    # 104 is the last variable: the coefficient of assettreat
    pvalue = reg.fit().pvalues[104]
    result.append(pvalue)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [6]:
table = df.groupby('assettreat')[control].mean().T  
table['p-value'] = result
table

assettreat,0.0,1.0,p-value
right_2013,0.245,0.241,0.964
left_2013,0.126,0.137,0.213
p_index_s1,0.004,0.051,0.399
e_index_init,-0.005,0.007,0.752
tradestock6all,0.368,0.355,0.290
male,0.513,0.521,0.470
age,41.530,39.289,0.011
postsecondary,0.232,0.230,0.953
BA_student,0.152,0.148,0.834
college_grad,0.427,0.426,0.860


In [7]:
#  Library to print professional publication
# tables in Latex, HTML, etc.
!pip install stargazer

Collum 1 of Table 1 shows that the estimate of intent to treat
(ITT) is 6.1%. It is worth mentioning that this type of experiment rarely has full compliance. Usually, when it is necessary to follow individuals over time,  there is an attrition rate for both the control and treatment groups.

Collum 2 shows that the ITT effect is robust to the addition of control variables and strata fixed effects.

Collum 3 presents the Weighted Least Squares (WLS) using the variable "vote_wgt" as weight. Jha & Shayo (2019) extracted a random sample, where the non-orthodox center voters were oversampled. The logic is to increase the precision for the most interesting group: swing voters. The use of weights allows reproducing the results without oversampling. 

Collum 4 shows that the results are driven by individuals in the treatment group that received Israeli stocks ("isrstock") and voucher to invest ("cash"). Be careful to conclude that Palestinian stocks have no effect. In the period of the experiment, the price of Israeli stocks increased but the Palestinian stocks decreased.

In [8]:
ITT1 = smf.ols("left_s3 ~ 1 + assettreat",
                 df).fit()

Xs = ['right_2013', 'left_2013', 'male', 'age', 'age2',
      'postsecondary', 'BA_student', 'college_grad',
      'married', 'tradestock6all', 'r_trad', 'r_relig',
      'r_ultra', 'g_jerusalem', 'g_north', 'g_haifa',
      'g_telaviv', 'g_south', 'g_wb', 'C(newses)',
      'willingrisk1to10', 'patient', 'plitscore']

controls = ""
for X in Xs:
    controls = controls + '+' + X      

ITT2 = smf.ols("left_s3 ~ 1 + assettreat" + controls +
                 "+C(block13)", df).fit()

WLS = smf.wls("left_s3 ~ 1 + assettreat" + controls +
       "+C(block13)", df, weights=df['vote_wgt']).fit()

treatments = "+ isrstock + palstock + cash"    
WLS2 = smf.wls("left_s3 ~ 1" + treatments + controls +
       "+C(block13)", df, weights=df['vote_wgt']).fit()

In [9]:
# Settings for a nice table
from stargazer.stargazer import Stargazer
stargazer = Stargazer([ITT1, ITT2, WLS, WLS2])

stargazer.title('Table 1 - Intent to Treat Estimates of Stock'
   + ' Exposure on Voting for a Left Party')

names = ['ITT', 'ITT', 'WLS', 'WLS']
stargazer.custom_columns(names, [1, 1, 1, 1])

stargazer.covariate_order(['assettreat', 'isrstock',
                           'palstock', 'cash'])

stargazer.add_line('Strata Fixed Effects', ['No', 'Yes',
                                            'Yes', 'Yes'])
stargazer.add_line('Covariates', ['No', 'Yes', 'Yes', 'Yes'])

stargazer

Column 1 of Table 2 presents the First Stage regression. The ITT ("assettreat") is the Instrumental Variable (IV) for compliance assignment ("asset_comp"). The variable "asset_comp" indicates who actually completed the experiment. After controlling for several covariates and Strata Fixed Effects, the coefficient of ITT is statistically significant. The variable "assettreat" is a perfect IV. This variable was randomized. Therefore, it is uncorrelated with the error term. 

Column 2 shows the result of the Control Function Approach (CF). The estimate of the treatment effect on the treated (TOT) is 7.3%. The treated individuals have the probability of voting for the left party increased by 7.3%. In this framework, the CF is equivalent to 2SLS. In the CF, we use the residual ($\hat{u}$) of the first stage to control for endogeneity in the second stage. Note that the residual is statistically significant. Therefore, correction is necessary. We can conclude that Table 1 underestimates the impact of financial asset exposure on voting for the left party.


In [10]:
# Fist Stage
FS = smf.ols("asset_comp ~ 1 + assettreat" + controls +
                 "+C(block13)", df).fit()

# Control Function Approach
df['resid'] = FS.resid
CF = smf.ols("left_s3 ~ 1 + asset_comp + resid" + controls +
                 "+C(block13)", df).fit()

In [11]:
# Settings for a nice table
stargazer = Stargazer([FS, CF])

stargazer.title('Table 2 - Impact of Stock Exposure'
 ' on Voting for a Left Party')

names = ['First Stage', 'Control Function']
stargazer.custom_columns(names, [1, 1])

stargazer.covariate_order(['assettreat', 'asset_comp', 'resid'])

stargazer.add_line('Strata Fixed Effects', ['Yes', 'Yes'])
stargazer.add_line('Covariates', ['Yes', 'Yes'])

stargazer 

## Exercises

1| The sample of Jha & Shayo (2019) is composed of Israeli voters. Speculate if the results would be qualitatively the same in the case of Palestinian voters. Justify your reasons.

2| What would be a promising research question that departs from Jha & Shayo (2019)?

3| What is social desirability bias? Describe what Jha & Shayo (2019) did to mitigate the social desirability bias.

4| Replicate the results of Table 2 using 2SLS rather than Control Function Approach. Do not use the library "linearmodels". Do the 2SLS manually using the library "statsmodels".

5| Do the exposure to Palestinian stocks decrease the probability of voting for right-wing parties? Run some regressions to justify your position.

## Reference

Jha, S. and Shayo, M. (2019). [Valuing Peace: The Effects of Financial Market Exposure on Votes and Political Attitudes](https://github.com/causal-methods/Papers/raw/master/Valuing%20Peace.pdf). Econometrica, 87: 1561-1588.

Montesquieu, C. (1748). [The Spirit of the Laws](https://oll.libertyfund.org/titles/montesquieu-complete-works-vol-1-the-spirit-of-laws). London: T. Evans, 1777, 4 vols. Vol. 2. Online Library of Liberty.